In [1]:
from PIL import Image, ImageDraw,ImageFont
import numpy as np


In [2]:
WIDTH, HEIGHT = 1080, 1080
def draw_circle(draw:ImageDraw, x, y, radius, c_fill, c_board, annot_class=0):
    ring_center = (x, y)
    outer_radius = radius
    inner_radius = outer_radius * 0.9
    ring_color = c_board
    fill_color = c_fill

    draw.ellipse(
    [(ring_center[0] - outer_radius, ring_center[1] - outer_radius),
     (ring_center[0] + outer_radius, ring_center[1] + outer_radius)],
    fill=ring_color)

    draw.ellipse(
    [(ring_center[0] - inner_radius, ring_center[1] - inner_radius),
     (ring_center[0] + inner_radius, ring_center[1] + inner_radius)],
    fill=fill_color)

    annotation = f"{annot_class} {x/WIDTH} {y/HEIGHT} {2*outer_radius/WIDTH} {2*outer_radius/HEIGHT} \n"
    return annotation

def draw_alpa_numeric(draw:ImageDraw, symbol:str, x, y, size=20, col=(0,0,0), f_path='RobotoSlab-VariableFont_wght.ttf'):
    font = ImageFont.truetype(f_path, size=size)
    draw.text((x,y), symbol, fill=col, font=font)


In [5]:
def is_intersection(obj1, obj2):
    """
    checks whether there is an intersection
    parameters:
    obj1: a tuple (x,y,r) x,y - centre coordinates and r - radius
    obj1: a tuple (x,y,r) x,y - centre coordinates and r - radius
    """
    diff = ((obj1[0] - obj2[0])**2 + (obj1[1] - obj2[1])**2)**0.5
    sr = obj1[2] + obj2[2]
    return diff <= sr

def generate_one_random( 
                 background_color=(255, 255, 255), #white, 
                 circle_fill_color = (155, 166, 168), #light-grey-green
                 circle_board_color = (43, 64, 83), #dark-gray-green
                 symbols = 'РЦЩАЗИЖКЕПЬЮЇДФГВЯОХСЙЄБЧУЛІНШТМҐ0123456789012345678901234567890123456789',
                 width = WIDTH,
                 height = HEIGHT
                ):
    img = Image.new('RGB', size=(width, height), color=background_color)
    draw = ImageDraw.Draw(img)

    padd_board = 30
    annotations = []
    objects = []
    for _ in range(30):
        flag = True
        while flag:
            x, y = np.random.randint(padd_board, width - padd_board), np.random.randint(padd_board, height - padd_board)
            r = np.random.randint(0.1*padd_board, padd_board)
            ob = (x, y, r)
            if sum([is_intersection(ob, other_ob) for other_ob in objects]) == 0:
                objects.append(ob)
                flag = False
                annotations.append(draw_circle(draw, x, y, r, c_fill=circle_fill_color, c_board=circle_board_color))

    for _ in range(10):
        flag = True
        while flag:
            x, y = np.random.randint(padd_board, width - padd_board), np.random.randint(padd_board, height - padd_board)
            r = np.random.randint(0.5*padd_board, padd_board)
            ob = (x, y, r)
            if sum([is_intersection(ob, other_ob) for other_ob in objects]) == 0:
                objects.append(ob)
                flag = False
                idx = np.random.randint(0, len(symbols))
                draw_alpa_numeric(draw, symbol=symbols[idx], x=x, y=y, size=r, col=circle_board_color)
    return img, annotations

def generate_one_block_max( 
                 background_color=(255, 255, 255), #white, 
                 circle_fill_color = (155, 166, 168), #light-grey-green
                 circle_board_color = (43, 64, 83), #dark-gray-green
                 width = WIDTH,
                 height = HEIGHT
                ):
    img = Image.new('RGB', size=(width, height), color=background_color)
    draw = ImageDraw.Draw(img)

    annotations = []
    padd_board = np.random.randint(5, 60)
    pad = np.random.randint(0, 7)
    """
    |-padd_board-|{|r-r||pad|...|r-r||pad|}|-padd_board-|
    """
    n_chairs = np.random.randint(5, 40)
    r = (width - 2 * padd_board -(n_chairs-1)*pad)/(2*n_chairs) 
    height_row = 2 * r + pad
    #max
    n_rows = int( height // height_row)
    dist_between_rows = (height - n_rows * height_row) / (n_rows + 1)

    for col in range(n_chairs):
        x = padd_board + pad + r + col * (2 * r + pad)
        for row in range(n_rows-2):
            y = padd_board//2 + r + dist_between_rows + row * (dist_between_rows + height_row) 
 
            annotations.append(draw_circle(draw, 
                                           x, 
                                           y, 
                                           r, 
                                           c_fill=circle_fill_color, 
                                           c_board=circle_board_color)
                                           )
    return img, annotations
    
def generate_one_block( 
                 background_color=(255, 255, 255), #white, 
                 circle_fill_color = (155, 166, 168), #light-grey-green
                 circle_board_color = (43, 64, 83), #dark-gray-green
                 width = WIDTH,
                 height = HEIGHT
                ):
    img = Image.new('RGB', size=(width, height), color=background_color)
    draw = ImageDraw.Draw(img)

    annotations = []
    padd_board = np.random.randint(5, 60)
    pad = np.random.randint(0, 7)
    """
    |-padd_board-|{|r-r||pad|...|r-r||pad|}|-padd_board-|
    """
    n_chairs = np.random.randint(5, 40)
    r = (width - 2 * padd_board -(n_chairs-1)*pad)/(2*n_chairs) 
    height_row = max([2 * r + pad, (padd_board + 1) // 2])
    n_rows_max = height // height_row
    n_rows = np.random.randint(2, n_rows_max)
    dist_between_rows = (height - n_rows * height_row) / (n_rows + 1)

    for col in range(n_chairs):
        x = padd_board + pad + r + col * (2 * r + pad)
        for row in range(n_rows):
            y = padd_board//2 + dist_between_rows + row * (dist_between_rows + height_row) 
            if col == 0:
                draw_alpa_numeric(draw, 
                                  symbol=str(row), 
                                  x=padd_board//2, 
                                  y=y-2*r, 
                                  size=2*r, 
                                  col=circle_board_color)
            if col == n_chairs - 1:
                draw_alpa_numeric(draw, 
                                  symbol=str(row), 
                                  x=width-padd_board//2, 
                                  y=y-2*r, 
                                  size=2*r, 
                                  col=circle_board_color)
            annotations.append(draw_circle(draw, 
                                           x, 
                                           y, 
                                           r, 
                                           c_fill=circle_fill_color, 
                                           c_board=circle_board_color)
                                           )
    return img, annotations
    
    



In [6]:
def generate_dataset(quantity=1500):
    imgs, annotations = [], []

    #create with white background
    for _ in range(quantity//4):
        img, annot = generate_one_random()
        imgs.append(img)
        annotations.append(annot)

    #create with white background
    for _ in range(quantity//4):
        img, annot = generate_one_block()
        imgs.append(img)
        annotations.append(annot)
    
    #create with white background but extremely dense 
    for _ in range(quantity//10):
        img, annot = generate_one_block_max()
        imgs.append(img)
        annotations.append(annot)

    #create with black background
    for _ in range(quantity//4):
        img, annot = generate_one_random(background_color=(0, 0, 0))
        imgs.append(img)
        annotations.append(annot)
    
    #create with black background
    for _ in range(quantity//4):
        img, annot = generate_one_block(background_color=(0, 0, 0))
        imgs.append(img)
        annotations.append(annot)
    
    #create with white background but extremely dense 
    for _ in range(quantity//10):
        img, annot = generate_one_block_max(background_color=(0, 0, 0))
        imgs.append(img)
        annotations.append(annot)
        
    return imgs, annotations

def save_dataset(imgs, annotations, images_path, labels_path):
    indices = np.random.permutation(len(imgs))
    for pos, idx in enumerate(indices):
        imgs[idx].save(images_path + f"{pos}.jpg")
        with open(labels_path + f"{pos}.txt", 'w') as file:
                file.writelines(annotations[idx])


In [7]:
# generate train dataset
imgs, annotations = generate_dataset(quantity=1500)
images_path = 'imgs/train/images/'
labels_path = 'imgs/train/labels/'
save_dataset(imgs, annotations, images_path, labels_path)

# generate valid dataset
imgs, annotations = generate_dataset(quantity=250)
images_path = 'imgs/valid/images/'
labels_path = 'imgs/valid/labels/'
save_dataset(imgs, annotations, images_path, labels_path)

# generate test dataset
imgs, annotations = generate_dataset(quantity=250)
images_path = 'imgs/test/images/'
labels_path = 'imgs/test/labels/'
save_dataset(imgs, annotations, images_path, labels_path)